<a href="https://colab.research.google.com/github/ZidanMau24/Chatbot-Batik/blob/main/QA_System_Batik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://2.bp.blogspot.com/-066qpJs0Ttc/WiYPXGNYEYI/AAAAAAAAFu8/XbOaf7DqfDMM9truu3DkrkIGfRgP4zBzgCLcBGAs/s1600/udinus.jpg"  width="200">

# AMS - QA System

oleh: Dr. Eng. Farrikh Alzami M.Kom; Abu Salam, M.Kom

disini kita akan belajar menggunakan modul:

dataset: (COVID19_wikipedia article.txt)

In [22]:
 from google.colab import drive
 drive.mount('/content/drive')
 import sys
 sys.path.append('/content/drive/My Drive/Colab Notebooks/QNA')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pathlib
pathlib.Path().resolve()

PosixPath('/content')

In [24]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
#importing the required packages
import nltk
#### Run it, if before never download nltk library
# nltk.download('all')
import re
import string
import gensim 

from gensim.models.fasttext import FastText
from gensim.parsing.preprocessing import remove_stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from nltk import pos_tag,word_tokenize,ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from nltk.corpus import wordnet,stopwords
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

**Uploading the Reference Article**

In [26]:
sample = '/content/drive/My Drive/Colab Notebooks/QNA/batik revisi.txt'


In [27]:
 #from google.colab import files
 #uploaded=files.upload()
sample = open("/content/drive/My Drive/Colab Notebooks/QNA/batik revisi.txt", "r") 

s = sample.read() 



In [28]:
s

'Batik adalah seni membuat kain dengan teknik pewarnaan khusus yang berasal dari Indonesia. Motif-motif batik dihasilkan dengan mengecat atau menorehkan lilin pada kain yang kemudian dicelupkan ke dalam pewarna.\n\nProses pembuatan batik biasanya dimulai dengan menggambar desain pada kain dengan menggunakan pensil. Kemudian, lilin dipanaskan hingga meleleh dan ditempelkan pada bagian kain yang tidak ingin diwarnai. Setelah itu, kain dicelupkan ke dalam pewarna, di mana warna hanya menempel pada bagian kain yang tidak tertutup lilin. Setelah kain diwarnai, lilin dihilangkan dengan memanaskannya atau dengan mencucinya.\n\nBatik memiliki berbagai macam motif dan warna yang berbeda, yang sering kali merefleksikan budaya dan kearifan lokal. Batik juga digunakan sebagai pakaian formal di Indonesia dan diakui sebagai warisan budaya dunia oleh UNESCO pada tahun 2009.\n\nSekarang, batik bukan hanya digunakan sebagai kain, tetapi juga diterapkan pada berbagai macam produk seperti tas, sepatu, ba

**Text Preprocessing Functions**

In [29]:
def stem_sentence(sentence):
  words=word_tokenize(sentence)
  #lemmatizer = WordNetLemmatizer()
  
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  new_words=[]
  for i in words:
    new_words.append(stemmer.stem(i))
    new_words.append(" ")
  return "".join(new_words)  
  


def clean_sentence(sentence, stopwords=True):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    
    
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
   
    
    return sentence
def get_cleaned_sentences(sents,stopwords=True):    
    
    cleaned_sentences=[]

    for i in sents:
        
        cleaning=clean_sentence(i,stopwords)
        cleaned=stem_sentence(cleaning)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences


**create_document_term_matrix: Function to create TF - IDF Matrix**


**calculate_cosine_similarity: Function to return the top 3 sentences based on cosine similarity between TF-IDF scores**

****

In [30]:
def create_document_term_matrix(sen,vectorizer):
  doc_term_matrix=vectorizer.fit_transform(sen)
  return DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names_out())
  

****

In [60]:
def calculate_cosine_similarity(df_list,sentences,question):
  a=[]
  for i in range(len(df_list)-1):
    sim=1 - spatial.distance.cosine(df_list[i], question)
    t=(sim,sentences[i])
    a.append(t)
  a.sort(reverse=True)
  n=[]
  for i in range(3):
    n.append(a[i][1])
    #print("*",n[i])
   
    
  return n  


**Function to classify questions**

In [32]:
def questiontype( question):
        questiontags = ['WP','WDT','WP$','WRB']
        question_POS = pos_tag(word_tokenize(question.lower()))
        
        question_Tags=[]
        for token in question_POS:
            if token[1] in questiontags:
              question_Tags.append(token)
                
                
        if len(question_Tags)==1 and question_Tags[0][0]!= 'apa' :
          return True
        else:
          return False  

**Function to find the most relevant sentence using n-gram similarity **

In [33]:
from nltk.util import ngrams
def n_gram_similarity(question,n):
  q=list(ngrams(word_tokenize(question.lower()),2))
  a=0
  b=0
  c=0
  t=[]
  for i in q:
    if i in list(ngrams(word_tokenize(n[0].lower()),2)):
      a=a+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[1].lower()),2)):
      b=b+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[2].lower()),2)):
      c=c+1        
  d=max(a,b,c)
  if a == d:
    t.append(n[0])
  if b == d:
    t.append(n[1]) 
  if c ==d:
    t.append(n[2])
  print()  
  print("Selected Sentence:",t[0])  
  return t  
  

**answertype:**Determines the type of entity that has to be returned, performs entity ranking if multiple entities a present

In [34]:
def answertype(question):
  nlp = spacy.load('en_core_web_sm')

  if (questiontype(question)):
    t='DESCRIPTIVE'
    flag=0
    word=word_tokenize(question.lower())
  
    if 'siapa' in word:
      t='PERSON'
    elif 'dimana' in word:
      t='GPE'
    elif 'berapa' in word and 'seberapa' in word and  'lama' in word or 'durasi' in word or 'panjang' in word or 'hari'in word or 'tahun' in word or'bulan' in word:
      t='DATE' 
    elif 'berapa' in word and 'banyak' in word :
       t = 'CARDINAL'  
    elif 'berapa' in word  or 'umur' in word or 'lama' in word or 'durasi' in word  or 'lama' in word or 'panjang' in word:
      t='DATE'
    elif 'berapa' in word  and 'panjang' in word or 'sering' or 'umur' in word or 'tahun' in word:
      t='DATE' 
    elif 'apa' in word and 'waktu' in word or 'durasi' in word or 'periode' in  'kata'  :
      t='DATE' 
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    n=n_gram_similarity(question,n)
    #print("Most relevant sentence", n[0])
    #print("ANSWER TYPE:",t)
    key = n[0]
    spdoc = nlp(key)
    entity_type=[]
    for ent in spdoc.ents:
       if ent.label_ == t:
          entity_type.append(ent.text)
    if len(entity_type) == 1:
      #print("ANSWER TYPE:", t)
      print("ANSWER:", entity_type[0])  
    if len(entity_type) == 0:
      #print("ANSWER TYPE:", t) 
      print(n[0])
    if len(entity_type) > 1:
      #print("Answer Type:",t)  
      key_question = question
      q=[]
      spdoc = nlp(key_question)
      for ent in spdoc:
        if ent.pos_ == 'NOUN' or ent.pos_ =='ADJ' :
          q.append(ent.text)
  
      key_answer = n[0]
      a = []
      spd = nlp(key)
      for ent in spd:
        if ent.pos_ == 'NOUN'or ent.pos_ =='ADJ' :
          a.append(ent.text)
  #s=[sentence.index(i) for i in t]
      s=[]
      w=[]
      for i in entity_type:
       s.append(n[0].index(i))
      for i in range(len(s)):
        w.append(0)

    
      for i in q:
        try:
           factor= n[0].index(i)
           for j in range(len(s)):
              w[j]=w[j]+(abs(s[j]-factor))
        except:
           continue    
      m=min(w)
      u=[]
      for i in range(len(s)):
        if w[i] == m:
           #print(entity_type[i])
           u.append(entity_type[i])
      print("ANSWER:",u[0])     
      

  

    
  else:
    t='DESCRIPTIVE'
    #print("ANSWER TYPE:",t)
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    #n = n_gram_similarity(question, n)
    for j in n:
         print(j) 

In [62]:
from textblob import TextBlob
from gensim.models.fasttext import FastText


question=["apa itu seni batik","apa saja motif batik"]

for j in question:

 
  qq=[]
  qp=[]
  que=sent_tokenize(j)
  
  qq.append(que)
  qp.append(j)
  questiontags = ['WP','WDT','WP$','WRB']
  question_POS = pos_tag(word_tokenize(j.lower()))
        
  question_Tags=[]
  for token in question_POS:
      if token[1] in questiontags:
          question_Tags.append(token)
  
 
  if len(question_Tags)>1:
     if ' and ' in j :
       pos=j.lower().find('and')
       qq=[]
       qp=[]
       qp.append(j[:pos])
       qp.append(j[pos+1:])
       qq.append(sent_tokenize(j[:pos])) 
       qq.append(sent_tokenize(j[pos+1:])) 
  
     
               
  

  
  print("QUESTION:",j)
  for k in range(len(qp)):   
       


    sentences=sent_tokenize(s)
    #q contains a list of cleaned sentence tokens of question
    q=get_cleaned_sentences(qq[k],stopwords=True)
    #preprocessed contains a list of cleaned sentence tokens of the reference text
    preprocessed=get_cleaned_sentences(sentences,stopwords=True)
  
    preprocessed.append(q[0])
    i=len(preprocessed)-1
    
  
    tfidf_vect=TfidfVectorizer()
    df=create_document_term_matrix(preprocessed,tfidf_vect) 
    df_list = df.values.tolist()
    answertype(qp[k])
    print()

QUESTION: apa itu seni batik
Batik adalah seni membuat kain dengan teknik pewarnaan khusus yang berasal dari Indonesia.
Setiap jenis batik memiliki keunikan dan nilai seni yang berbeda-beda, tergantung pada motif, warna, dan teknik pembuatannya.
Selain itu, batik juga terus menjadi bagian penting dari budaya Indonesia dan dihargai oleh banyak orang di seluruh dunia.

QUESTION: apa saja motif batik
Setiap daerah di Indonesia memiliki motif batik yang khas dan unik.
Beberapa contoh motif batik yang terkenal adalah Batik Parang dari Yogyakarta, Batik Lasem dari Jawa Tengah, dan Batik Tulis dari Solo.
Batik Tulis - merupakan jenis batik yang dibuat secara tradisional dengan teknik menulis atau menorehkan motif pada kain menggunakan lilin cair.

